# Data Science with Python Course Exam Intermediate level

On this activity we work with the receipt's data of a papercraft company called "DM", we will integrate this data with the product dataset of the company to extract valuable information.

In [1]:
%pip install -r ../requirements.txt

import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Data cleansing

### 1. Load dataset


In [2]:
# 1. Load the dataset
input_file = '../data/detalle_boletas.csv'

detalle_boletas = pd.read_csv(input_file,sep=',', encoding='utf-8')
# Display the first 5 rows of the dataset to check if it was loaded correctly
print(detalle_boletas.head())

print(detalle_boletas.dtypes)

          Fecha      ID          NXXX  Cantidad  Precio_prod
0      2017/1/1  400005  554170000001         4         9808
1  2017!/{1-/.1  400002  554170000001         3         1355
2      2017/1/1  400001  554170000001         4         8756
3      2017/1/1  400008  554170000001         1         9004
4      2017/1/1  400001  554170000002         1         6736
Fecha          object
ID             object
NXXX           object
Cantidad        int64
Precio_prod     int64
dtype: object


### 2. Modify dataset

Before to begin the analysis, it is necesary to modify the dataset ```detalle_boletas```. Particularly:

a. Delete the column ```Precio_prod``` because the prices are incorrect.

b. Create a column ```Pais_Venta```, because the company has intention to get into the international market in the near future. For now all the values of this column should be "Chile".

c. Change column name ```NXXX``` to ```Num Boleta``` to make it more descriptive.

You can notice, by the arrangement of the data in the dataset, when a receipt has more than 1 product then it could be several rows refering to that receipt.

![alt text](../images/receipts.png)

In this case:

- The receipt 554170000002 has two products, with ```ID``` 400009 and ```ID``` 400007. 3 and 2 units of each product were sold, respectively. It was issued on January 1, 2016.
- The receipt 554170000003 has one product, with ```ID``` 400005. 2 units of this product were sold. It was issued on January 2, 2016.
- The receipt 554170000004 has three products, with ```ID``` 400005, ```ID``` 400001, and ```ID``` 400002. 2 units of each product were sold. It was issued on January 2, 2016.


Some important considerations:

- One or more than one receipt could be issued by day.
- A receipt will never have 2 different issuance date.
- It will never exist 2 row of the same receipt and same product


In [3]:
detalle_boletas = detalle_boletas.drop(columns=['Precio_prod'])
print(detalle_boletas.head())

detalle_boletas['Pais_Venta'] = 'Chile'
print(detalle_boletas.head())

detalle_boletas = detalle_boletas.rename(columns={'NXXX': 'Num Boleta'})
print(detalle_boletas.head())

          Fecha      ID          NXXX  Cantidad
0      2017/1/1  400005  554170000001         4
1  2017!/{1-/.1  400002  554170000001         3
2      2017/1/1  400001  554170000001         4
3      2017/1/1  400008  554170000001         1
4      2017/1/1  400001  554170000002         1
          Fecha      ID          NXXX  Cantidad Pais_Venta
0      2017/1/1  400005  554170000001         4      Chile
1  2017!/{1-/.1  400002  554170000001         3      Chile
2      2017/1/1  400001  554170000001         4      Chile
3      2017/1/1  400008  554170000001         1      Chile
4      2017/1/1  400001  554170000002         1      Chile
          Fecha      ID    Num Boleta  Cantidad Pais_Venta
0      2017/1/1  400005  554170000001         4      Chile
1  2017!/{1-/.1  400002  554170000001         3      Chile
2      2017/1/1  400001  554170000001         4      Chile
3      2017/1/1  400008  554170000001         1      Chile
4      2017/1/1  400001  554170000002         1      Chile


### 3. Cleansing


The previous file ```detalle_boleta.csv``` was dirty and it should be cleaned

a. There are some products with ID "4XXXXXX" y Num Boleta "55417XXXXXXX". Delete any row in the dataset that contains these values, becuase they were not correctly generated by the system and it should not be considered in the analysis.

b. The column Fecha has extra characters. Clean the column in order to obtain the format YYYY/MM/DD (without extra characters). Particularly, identify which extra characters exist in the column besides "/" or numbers and remove them.

In [4]:
filter = (detalle_boletas['ID'].str.contains('4XXXXX')) | (detalle_boletas['Num Boleta'].str.contains('55417XXXXXXX'))
print("Rows to be removed based on filter:")
print(detalle_boletas[filter])


detalle_boletas = detalle_boletas.drop(detalle_boletas[filter].index)
print(detalle_boletas.head())


# Identified extra characters in the 'Fecha' column
# The characters to be removed are: '{', '.', '_', '-', '!'
# Replaced with empty string and recognized with regex
detalle_boletas['Fecha'] = detalle_boletas['Fecha'].str.replace(r'[{._\-!]', '', regex=True)
print(detalle_boletas.head())


Rows to be removed based on filter:
              Fecha      ID    Num Boleta  Cantidad Pais_Venta
7          2017/1/2  4XXXXX  55417XXXXXXX         2      Chile
8      2017!/{1-/.2  400009  55417XXXXXXX         5      Chile
9          2017/1/2  400010  55417XXXXXXX         1      Chile
12         2017/1/3  4XXXXX  554170000004         4      Chile
16        2017_/1/3  400001  55417XXXXXXX         4      Chile
...             ...     ...           ...       ...        ...
4404  2018!/{8-/.29  400003  55417XXXXXXX         2      Chile
4409  2018!/{8-/.29  4XXXXX  554170001746         3      Chile
4411      2018/8/30  4XXXXX  554170001747         3      Chile
4417      2018/8/30  4XXXXX  554170001748         1      Chile
4419     2018_/8/30  4XXXXX  554170001749         4      Chile

[651 rows x 5 columns]
          Fecha      ID    Num Boleta  Cantidad Pais_Venta
0      2017/1/1  400005  554170000001         4      Chile
1  2017!/{1-/.1  400002  554170000001         3      Chile
2      

## Data extraction

### 4. Descriptive statistics Cantidad

Calculate the descriptive statistics of the column Cantidad for each existing product and print it to the console. The descriptive statistics should include mean, standard deviation, minimum and maximum values. See image below.


![Example descriptive statistic](../images/descriptive_statistic_example.png)

In [16]:
pivot_table_boletas = detalle_boletas.pivot_table(index=['ID'], values=['Cantidad'], aggfunc={np.amax, np.amin, np.mean, np.std})
print(pivot_table_boletas.head(10))

       Cantidad                         
           amax amin      mean       std
ID                                      
400001        5    1  3.005435  1.431439
400002        5    1  2.953125  1.420804
400003        5    1  2.955112  1.409957
400004        5    1  3.016484  1.429617
400005        5    1  3.036554  1.430308
400006        5    1  3.087432  1.368133
400007        5    1  3.008523  1.466603
400008        5    1  3.040921  1.440571
400009        5    1  3.107330  1.390443
400010        5    1  2.916667  1.430125


/var/folders/y4/6837m289589_j2ngnfmktk5r0000gn/T/ipykernel_99309/1649877467.py:1: FutureWarning: The provided callable <function mean at 0x109844ea0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pivot_table_boletas = detalle_boletas.pivot_table(index=['ID'], values=['Cantidad'], aggfunc={np.amax, np.amin, np.mean, np.std})
/var/folders/y4/6837m289589_j2ngnfmktk5r0000gn/T/ipykernel_99309/1649877467.py:1: FutureWarning: The provided callable <function std at 0x109844fe0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  pivot_table_boletas = detalle_boletas.pivot_table(index=['ID'], values=['Cantidad'], aggfunc={np.amax, np.amin, np.mean, np.std})


### 5. Data extraction: Separate Fecha column

Now that the information is clean on detalle_boletas, generate column ```Anho``` (with the year of the column ```Fecha```) and a column ```Mes``` (with the month of the column ```Fecha```) and a column ```Dia``` (with the day of the column ```Fecha```). This columns should be added to the dataframe detalle_boletas. Then, delete the column ```Fecha```.


In [6]:
separated_date = detalle_boletas['Fecha'].str.split('/', expand=True)
print(separated_date.head())


separated_date.columns = ['Anho', 'Mes', 'Dia']
detalle_boletas = detalle_boletas.join(separated_date)
print(detalle_boletas.head())

detalle_boletas= detalle_boletas.drop(columns=['Fecha'])
print(detalle_boletas.head())

      0  1  2
0  2017  1  1
1  2017  1  1
2  2017  1  1
3  2017  1  1
4  2017  1  1
      Fecha      ID    Num Boleta  Cantidad Pais_Venta  Anho Mes Dia
0  2017/1/1  400005  554170000001         4      Chile  2017   1   1
1  2017/1/1  400002  554170000001         3      Chile  2017   1   1
2  2017/1/1  400001  554170000001         4      Chile  2017   1   1
3  2017/1/1  400008  554170000001         1      Chile  2017   1   1
4  2017/1/1  400001  554170000002         1      Chile  2017   1   1
       ID    Num Boleta  Cantidad Pais_Venta  Anho Mes Dia
0  400005  554170000001         4      Chile  2017   1   1
1  400002  554170000001         3      Chile  2017   1   1
2  400001  554170000001         4      Chile  2017   1   1
3  400008  554170000001         1      Chile  2017   1   1
4  400001  554170000002         1      Chile  2017   1   1


## Data integration

### 6. Load Products

Load the file ```Lista productos.csv``` as a data frame and name it ```lista_productos```.

This file contains the detail of the 10 products available in stock.

![alt text](../images/products.png)

Where:
- ```ID```: Identifier of each product
- ```Nombre```: Product's name
- ```Descrip```: Product's description
- ```Precio Unitario```: Product's unit price

In [9]:
input_file_productos = '../data/Lista productos.csv'

lista_productos = pd.read_csv(input_file_productos, sep=',', encoding='utf-8')
print(lista_productos.head(10))

print(lista_productos.dtypes)


       ID                 Nombre  \
0  400001             Alerce A4C   
1  400002             Alerce A4O   
2  400003          Alerce A4C XL   
3  400004          Alerce A4O XL   
4  400005           Alerce Kraft   
5  400006        Alerce Kraft XL   
6  400007  Alerce PreCuad Oficio   
7  400008   Alerce PreCuad Carta   
8  400009  Alerce PreComp Oficio   
9  400010   Alerce PreComp Carta   

                                         Descripción  Precio Unitario  
0                           Resma A4 Carta 500 hojas             2250  
1                          Resma A4 Oficio 500 hojas             2500  
2                          Resma A4 Carta 1000 hojas             4200  
3                         Resma A4 Oficio 1000 hojas             4700  
4                         Pliego papel kraft 90x60cm              500  
5                        Pliego papel kraft 150x90cm              750  
6  Block prepicado, tamaño oficio, cuadriculado, ...             1300  
7  Block prepicado, tamaño 

### 7. Merge dataframes

Join the DataFrame ```lista_productos``` with the DataFrame ```detalle_boletas```, based on the information in the ```ID``` column.

The resulting DataFrame from this join must contain the same information as the DataFrame ```detalle_boletas```, but now each row must also include the product name, the description, and the unit price.

You must call this DataFrame ```detalle_boletas2```. Print this DataFrame to the console.

Pay attention to the data type, because in order to perform this join, the column used to match values in both DataFrames must have the same type.

In [11]:
lista_productos['ID'] = lista_productos['ID'].astype(str)

detalle_boletas2 = detalle_boletas.merge(lista_productos, on='ID', how='left')
print(detalle_boletas2.head(20))

        ID    Num Boleta  Cantidad Pais_Venta  Anho Mes Dia  \
0   400005  554170000001         4      Chile  2017   1   1   
1   400002  554170000001         3      Chile  2017   1   1   
2   400001  554170000001         4      Chile  2017   1   1   
3   400008  554170000001         1      Chile  2017   1   1   
4   400001  554170000002         1      Chile  2017   1   1   
5   400002  554170000002         5      Chile  2017   1   1   
6   400005  554170000002         2      Chile  2017   1   1   
7   400003  554170000004         4      Chile  2017   1   3   
8   400009  554170000004         5      Chile  2017   1   3   
9   400006  554170000005         3      Chile  2017   1   3   
10  400003  554170000005         3      Chile  2017   1   3   
11  400001  554170000005         1      Chile  2017   1   3   
12  400007  554170000007         1      Chile  2017   1   4   
13  400007  554170000008         1      Chile  2017   1   4   
14  400002  554170000008         3      Chile  2017   1

### 8. Total Revenue

Calculate how much revenue each receipt (boleta) generated from the sale of products.
To do this, add a new column named ```Ingreso total``` to the DataFrame ```detalle_boletas2```.
This column must contain the values resulting from multiplying the ```Precio Unitario``` column by the ```Cantidad ```column.
Print the DataFrame ```detalle_boletas2``` with this new column to the console.

In [14]:
ingreso_total = lambda x: x['Cantidad'] * x['Precio Unitario']

detalle_boletas2['Ingreso total'] = ingreso_total(detalle_boletas2)
print(detalle_boletas2.head())

       ID    Num Boleta  Cantidad Pais_Venta  Anho Mes Dia  \
0  400005  554170000001         4      Chile  2017   1   1   
1  400002  554170000001         3      Chile  2017   1   1   
2  400001  554170000001         4      Chile  2017   1   1   
3  400008  554170000001         1      Chile  2017   1   1   
4  400001  554170000002         1      Chile  2017   1   1   

                 Nombre                                        Descripción  \
0          Alerce Kraft                         Pliego papel kraft 90x60cm   
1            Alerce A4O                          Resma A4 Oficio 500 hojas   
2            Alerce A4C                           Resma A4 Carta 500 hojas   
3  Alerce PreCuad Carta  Block prepicado, tamaño carta, cuadriculado, 8...   
4            Alerce A4C                           Resma A4 Carta 500 hojas   

   Precio Unitario  Ingreso total  
0              500           2000  
1             2500           7500  
2             2250           9000  
3             

### 9. Descriptive statistic Total Revenue

Finally, calculate descriptive statistics of the column ```Ingreso total``` for each of the products that exist.
The descriptive statistics you must calculate are: mean (media), standard deviation (desviación estándar), minimum (mínimo), and maximum (máximo).
Your result should look like this:

![Descriptive statistics example 2](../images/descriptive_statistic_example_2.png)

In [15]:
descriptive_statistic = detalle_boletas2.pivot_table(index=['ID'], values=['Ingreso total'], aggfunc={np.amax, np.amin, np.mean, np.std, np.sum})
print(descriptive_statistic.head(10))

       Ingreso total                                          
                amax  amin          mean          std      sum
ID                                                            
400001         11250  2250   6762.228261  3220.737038  2488500
400002         12500  2500   7382.812500  3552.010633  2835000
400003         21000  4200  12411.471322  5921.821348  4977000
400004         23500  4700  14177.472527  6719.199758  5160600
400005          2500   500   1518.276762   715.153963   581500
400006          3750   750   2315.573770  1026.099450   847500
400007          6500  1300   3911.079545  1906.583877  1376700
400008          5500  1100   3345.012788  1584.627899  1307900
400009          6000  1200   3728.795812  1668.531696  1424400
400010          5000  1000   2916.666667  1430.124731  1120000


/var/folders/y4/6837m289589_j2ngnfmktk5r0000gn/T/ipykernel_99309/1948404911.py:1: FutureWarning: The provided callable <function sum at 0x10982ba60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  descriptive_statistic = detalle_boletas2.pivot_table(index=['ID'], values=['Ingreso total'], aggfunc={np.amax, np.amin, np.mean, np.std, np.sum})
/var/folders/y4/6837m289589_j2ngnfmktk5r0000gn/T/ipykernel_99309/1948404911.py:1: FutureWarning: The provided callable <function mean at 0x109844ea0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  descriptive_statistic = detalle_boletas2.pivot_table(index=['ID'], values=['Ingreso total'], aggfunc={np.amax, np.amin, np.mean, np.std, np.sum})
/var/folders/y4/6837m289589_j2ngnfmktk5r0000gn/T/ipykernel_9930